<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px" alt="Vera C. Rubin Observatory Logo"> 
<h1 style="margin-top: 10px">Retrieve and Aggregate Zooniverse Output</h1>
Authors: Becky Nevin, Clare Higgs, and Eric Rosas <br>
Contact author: Clare Higgs <br>
Last verified to run: 2024-10-07 <br>
LSST Science Pipelines version: Weekly 2024_37 <br>
Container size: small or medium <br>
Targeted learning level: intermediate

<b>Description:</b> This notebook guides a PI through the process of retrieving classification data from Zooniverse and builds upon Hayley Robert's Aggregation notebook example. <br><br>
<b>Skills:</b> Query for Zooniverse classification data via the panoptes client; retrieve and aggregate user classifications and retrieve original objectIds or diaobjectIds.
<br><br>
<b>LSST Data Products:</b> n/a<br><br>
<b>Packages:</b> panoptes_client, panoptes_aggregation, rubin.citsci, utils (citsci plotting and display utilities),  <br><br>
<b>Credit:</b> Hayley Roberts' aggregation code https://github.com/astrohayley/SLSN-Aggregation-Example/blob/main/SLSN_batch_aggregation.py<br><br>
<b>Get Support: </b>PIs new to DP0 are encouraged to find documentation and resources at <a href="https://dp0-2.lsst.io/">dp0-2.lsst.io</a>. Support for this notebook is available and questions are welcome at cscience@lsst.org.

## 1. Introduction <a class="anchor" id="first-bullet"></a>
This notebook provides an introduction to how to use the Zooniverse panoptes client and rubin.citsci package to retrieve classifications from Zooniverse and aggregate the results. Data aggregation in this context is collecting classifications across all citizen scientists and summarizing them by subject in terms of classifier majority.

### 1.1 Package imports <a class="anchor" id="second-bullet"></a>

#### Install Pipeline Package

First, install the Rubin Citizen Science Pipeline package by doing the following:

1. Open up a New Launcher tab
2. In the "Other" section of the New Launcher tab, click "Terminal"
3. Use `pip` to install the `rubin.citsci` package by entering the following command:
```
pip install rubin.citsci
```
Note that this package will soon be installed directly on RSP.

If this package is already installed, make sure it is updated:
```
pip install --u rubin.citsci
```

4. Confirm the next cell containing `from rubin.citsci import pipeline` works as expected and does not throw an error

5. Install panoptes_client:
```
pip install panoptes_client
pip install panoptes_aggregation
```

6. If the pip install doesn't work for `panoptes_aggregation`:
```
pip install -U git+git://github.com/zooniverse/aggregation-for-caesar.git
```
(https://www.zooniverse.org/talk/1322/2415041?comment=3969837&page=1)

In [10]:
import numpy as np
import pandas as pd
import getpass
import json
import os

# Zooniverse tools
from panoptes_client import Panoptes, Workflow
from panoptes_aggregation.extractors.utilities import annotation_by_task
from panoptes_aggregation.extractors import question_extractor
from panoptes_aggregation.reducers import question_consensus_reducer
from panoptes_client.panoptes import PanoptesAPIException

from tqdm import tqdm

from rubin.citsci import pipeline

### 1.2 Define functions and parameters <a class="anchor" id="third-bullet"></a>
Credit for these functions goes to Hayley Roberts at Zooniverse. This includes:
- `download_classifications`: A function to download the classifications given a workflow ID in, which returns a dataframe
- `extract_data`: A function to extract user annotations by task and sort by when they were classified??? This can be modified for other classification tasks such as drawing, please see the Zooniverse documentation.
- `aggregate_data`: A function that groups by task and user, selects the most recent classification from each user, and uses the Zooniverse `question_consesus_reducer` function to determine the consensus for each subject ID amongst all users

In [59]:
def download_classifications(WORKFLOW_ID, client, generate):
    """
    Downloads data from Zooniverse

    Args:
        WORKFLOW_ID (int): Workflow ID of workflow being aggregated
        client: Logged in Zooniverse client
        generate (bool): Options are True or False
                         Use generate=True means when you want to
                         generate a new classification export. Use this
                         option if there are new user classifications
                         to include since the previous time the exports
                         were downloaded. generate=True is also needed if
                         downloading the classifications for the first time.
                         Use generate=False when you wish to re-use a
                         previously generated export, this also saves
                         significant computational time, as Zooniverse
                         does not need to re-generate the export on a
                         row-by-row basis.

    Returns:
        classification_data (DataFrame): Raw classifications from Zooniverse
    """
    print('beginning function')
    workflow = Workflow(WORKFLOW_ID)
    with client:
        try:
            classification_export = workflow.get_export('classifications',
                                                        generate=generate,
                                                        wait=False)
        
        except PanoptesAPIException:
            # This error will be thrown if no classifications exist and 
            # its necessary to set generate to True. This will happen
            # the first time you're downloading the classifications
            # from a workflow.
            classification_export = workflow.get_export('classifications',
                                                    generate=True,
                                                    wait=False)
        
        # since it's a partial class, call it to get the DictReader object
        csv_dictreader_instance = classification_export.csv_dictreader()
        try:
            classification_rows = [row for row in tqdm(csv_dictreader_instance, file=sys.stdout)]
        except:
            # this will happen if there's an error in the download
            classification_export = workflow.get_export('classifications',
                                                    generate=True,
                                                    wait=False)
            csv_dictreader_instance = classification_export.csv_dictreader()
            classification_rows = [row for row in tqdm(csv_dictreader_instance, file=sys.stdout)]
            
    # convert to pandas dataframe
    classification_data = pd.DataFrame.from_dict(classification_rows)
    return classification_data



def extract_data(classification_data, id_type='#objectId'):
    """
    Extracts annotations from the classification data

    Args:
        classification_data (DataFrame): Raw classifications from Zooniverse
        id_type (str): Name of the id in the extracted classifications; the #
                       default that precedes objectId keeps the objectId hidden
                       from the user in the information window on Zooniverse.

    Returns:
        extracted_data (DataFrame): Extracted annotations from raw classification data
    """
    # set up our list where we will store the extracted data temporarily
    extracted_rows = []
    # iterate through our classification data
    for i in range(len(classification_data)):
        # access the specific row and extract the annotations
        row = classification_data.iloc[i]
        for annotation in json.loads(row.annotations):
            row_annotation = annotation_by_task({'annotations': [annotation]})
            extract = question_extractor(row_annotation)
            subject_id_str = str(row.subject_ids)
            # Check if the subject ID exists and is a dictionary
            if subject_id_str in row.subject_data and isinstance(json.loads(row.subject_data)[subject_id_str], dict):
                rubin_id = json.loads(row.subject_data)[subject_id_str][id_type]
            else:
                print(f"Key '{subject_id_str}' not found in subject_data or it is not a dictionary.")
            # add the extracted annotations to our temporary list along with some other additional data
            extracted_rows.append({
                'classification_id': row.classification_id,
                'subject_id':        row.subject_ids,
                'user_name':         row.user_name,
                'user_id':           row.user_id,
                'created_at':        row.created_at,
                'rubin_id':          rubin_id,
                'data':              json.dumps(extract),
                'task':              annotation['task']
            })
    # convert the extracted data to a pandas dataframe and sort
    extracted_data = pd.DataFrame.from_dict(extracted_rows)
    extracted_data.sort_values(['subject_id', 'created_at'], inplace=True)
    return extracted_data

def last_filter(data):
    """
    Determines the most recently submitted classifications
    """
    last_time = data.created_at.max()
    ldx = data.created_at == last_time
    return data[ldx]

def aggregate_data(extracted_data):
    """
    Aggregates question data from extracted annotations

    Args:
        extracted_data (DataFrame): Extracted annotations from raw classifications

    Returns:
        aggregated_data (DataFrame): Aggregated data for the given workflow
    """
    # generate an array of unique subject ids - these are the ones that we will iterate over
    subject_ids_unique = np.unique(extracted_data.subject_id)

    # Create a dictionary to map subject IDs to their corresponding metadata
    rubin_ids_unique = extracted_data.groupby('subject_id')['rubin_id'].unique()

    '''
    print('len subject-ids-unique', len(subject_ids_unique))
    print('metadata saved', len(rubin_id))
    print(rubin_id)
    STOP
    '''
    

    # set up a temporary list to store reduced data
    aggregated_rows = []

    # determine the total number of tasks
    tasks = np.unique(extracted_data.task)

    # iterating over each unique subject id
    for i in range(len(subject_ids_unique)):

        # determine the subject_id to work on
        subject_id = subject_ids_unique[i]
        rubin_id = rubin_ids_unique.iloc[i][0]

        # filter the extract_data dataframe for only the subject_id that is being worked on
        extract_data_subject = extracted_data[extracted_data.subject_id==subject_id].drop_duplicates()

        for task in tasks:

            extract_data_filtered = extract_data_subject[extract_data_subject.task == task]

            # if there are less unique user submissions than classifications, filter for the most recently updated classification
            if (len(extract_data_filtered.user_name.unique()) < len(extract_data_filtered)):
                extract_data_filtered = extract_data_filtered.groupby(['user_name'], group_keys=False).apply(last_filter)

            # iterate through the filtered extract data to prepare for the reducer
            classifications_to_reduce = [json.loads(extract_data_filtered.iloc[j].data) for j in range(len(extract_data_filtered))]

            # use the Zooniverse question_consesus_reducer to get the final consensus
            # WHAT ARE THE ARGUMENTS THAT ARE OPTIONAL HERE?
            reduction = question_consensus_reducer(classifications_to_reduce)

            # add the subject id to our reduction data
            reduction['subject_id'] = subject_id
            reduction['task'] = task
            reduction['rubin_id'] = rubin_id

            # add the data to our temporary list
            aggregated_rows.append(reduction)


    # converting the result to a dataframe
    aggregated_data = pd.DataFrame.from_dict(aggregated_rows)

    # drop rows that are nan
    aggregated_data.dropna(inplace=True)

    return aggregated_data

## 2. Log into Zooniverse and find the workflow to download classifications from
If you're running this notebook, you should already have a Zooniverse account with a project with classifications. If you do not yet have an account, please return to notebook `01_Introduction_to_Citsci_Pipeline.ipynb`.

IMPORTANT: Your Zooniverse project must be set to "public", a "private" project will not work. Select this setting under the "Visibility" tab, (it does not need to be set to live). 

Supply the email associated with your Zooniverse account, and then follow the instructions in the prompt to log in and select your project by slug name. 

A "slug" is the string of your Zooniverse username and your project name without the leading forward slash, for instance: "username/project-name". [Click here for more details](https://www.zooniverse.org/talk/18/967061?comment=1898157&page=1).

**The `rubin.citsci` package includes a method that creates a Zooniverse project from template. If you wish to use this feature, do not provide a slug_name and run the subsequent cell.**

In [3]:
email = "beckynevin@gmail.com"
cit_sci_pipeline = pipeline.CitSciPipeline()
cit_sci_pipeline.login_to_zooniverse(email)

Loading and running utilities to establish a link with Zooniverse
Enter your Zooniverse username followed by password below
Enter your Zooniverse credentials...


Username:  rebecca.nevin
 ········


You now are logged in to the Zooniverse platform.

*==* Your Project Slugs *==*

rebecca-dot-nevin/template-test-copy-2024-07-09-21-49-53
rebecca-dot-nevin/template-test-copy-2024-07-09-19-02-02
rebecca-dot-nevin/template-test-copy-2024-07-09-18-53-39
rebecca-dot-nevin/template-test-copy-2024-07-03-21-54-30
rebecca-dot-nevin/template-test-copy-2024-07-02-22-11-52
rebecca-dot-nevin/pcw-2023-awesome-citsci-project
rebecca-dot-nevin/test-project
rebecca-dot-nevin/galaxy-rotation-fields

*==========================*



Which project would you like to connect to? (copy & paste the slug name here)? rebecca-dot-nevin/test-project


Current project set to: rebecca-dot-nevin/test-project


Use the `list_workflows` method to find the workflow ID.

In [4]:
cit_sci_pipeline.list_workflows()


*==* Your Workflows *==*

Workflow ID: 27592 - Display Name: Classify with hidden metadata double backslash before objectId
Workflow ID: 27550 - Display Name: Classify with hidden metadata backslash
Workflow ID: 27568 - Display Name: Classify with hidden metadata hashtag before objectId
Workflow ID: 27546 - Display Name: Classify with hidden metadata
Workflow ID: 27509 - Display Name: Classify with metadata
Workflow ID: 23254 - Display Name: Classification

*==========================*



Copy and paste the above ID into the cell below.

In [43]:
WORKFLOW_ID = 27568

## 3. Download the classifications
These will still be in the raw format. This function reads from the output csv and puts all rows into a dataframe format.

PanoptesAPIException: No classifications_export exists for workflow #27550

In [60]:
client = cit_sci_pipeline.client
# how long should this take?
classification_data = download_classifications(WORKFLOW_ID, client, generate=False)

beginning function
36it [00:00, 24085.97it/s]


In [61]:
classification_data

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,591515226,rebecca.nevin,1946584,16e3101aa7c591add961,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:25 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822135"":{""retired"":null,""#objectId"":""1651...",103822135
1,591515240,rebecca.nevin,1946584,b0221feeeb5c232b95af,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:38 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822131"":{""retired"":null,""#objectId"":""1567...",103822131
2,591515244,rebecca.nevin,1946584,b0221feeeb5c232b95af,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:39 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822134"":{""retired"":null,""#objectId"":""1651...",103822134
3,591515247,rebecca.nevin,1946584,b0221feeeb5c232b95af,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:42 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822132"":{""retired"":null,""#objectId"":""1650...",103822132
4,591515250,rebecca.nevin,1946584,4951e699da8734745854,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:43 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822133"":{""retired"":null,""#objectId"":""1651...",103822133
5,591515254,rebecca.nevin,1946584,4951e699da8734745854,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:45 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822133"":{""retired"":null,""#objectId"":""1651...",103822133
6,591515256,rebecca.nevin,1946584,4951e699da8734745854,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:47 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822131"":{""retired"":null,""#objectId"":""1567...",103822131
7,591515260,rebecca.nevin,1946584,d6752a240ee7569a5c12,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:49 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822135"":{""retired"":null,""#objectId"":""1651...",103822135
8,591515263,rebecca.nevin,1946584,d6752a240ee7569a5c12,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:51 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822134"":{""retired"":null,""#objectId"":""1651...",103822134
9,591515265,rebecca.nevin,1946584,d6752a240ee7569a5c12,27568,Classify with hidden metadata hashtag before o...,4.8,2024-10-17 02:43:52 UTC,,,"{""source"":""api"",""session"":""1449ea4e40d434103ea...","[{""task"":""T0"",""task_label"":""Is this a galaxy?""...","{""103822132"":{""retired"":null,""#objectId"":""1650...",103822132


In [46]:
classification_data['subject_data'][0]

'{"103822135":{"retired":null,"#objectId":"1651325727431231924"}}'

Select either `objectId` or `diaObjectId`; this should match the ID type of the data that was first sent to Zooniverse.

## 4. Extract annotations by task and sort by subject ID
The `id_type` argument should either be set to 'objectId' (default) or 'diaobjectId'. This function will return all annotations, there are repeated rows for some `subject_id` entries from different users or the same user re-classifying the same subject. This function will also return the Rubin IDs in a table.

In [55]:
extracted_data = extract_data(classification_data, id_type='#objectId')
extracted_data

,classification_id,subject_id,user_name,user_id,created_at,rubin_id,data,task
1,591515240,103822131,rebecca.nevin,1946584,2024-10-17 02:43:38 UTC,1567965153859768169,"{""yes"": 1, ""aggregation_version"": ""4.1.0""}",T0
6,591515256,103822131,rebecca.nevin,1946584,2024-10-17 02:43:47 UTC,1567965153859768169,"{""no"": 1, ""aggregation_version"": ""4.1.0""}",T0
13,591515273,103822131,rebecca.nevin,1946584,2024-10-17 02:43:59 UTC,1567965153859768169,"{""no"": 1, ""aggregation_version"": ""4.1.0""}",T0
20,591515292,103822131,rebecca.nevin,1946584,2024-10-17 02:44:12 UTC,1567965153859768169,"{""yes"": 1, ""aggregation_version"": ""4.1.0""}",T0
25,591515306,103822131,rebecca.nevin,1946584,2024-10-17 02:44:20 UTC,1567965153859768169,"{""yes"": 1, ""aggregation_version"": ""4.1.0""}",T0
31,591515324,103822131,rebecca.nevin,1946584,2024-10-17 02:44:29 UTC,1567965153859768169,"{""no"": 1, ""aggregation_version"": ""4.1.0""}",T0
3,591515247,103822132,rebecca.nevin,1946584,2024-10-17 02:43:42 UTC,1650947495431285770,"{""yes"": 1, ""aggregation_version"": ""4.1.0""}",T0
9,591515265,103822132,rebecca.nevin,1946584,2024-10-17 02:43:52 UTC,1650947495431285770,"{""yes"": 1, ""aggregation_version"": ""4.1.0""}",T0
16,591515280,103822132,rebecca.nevin,1946584,2024-10-17 02:44:05 UTC,1650947495431285770,"{""yes"": 1, ""aggregation_version"": ""4.1.0""}",T0
18,591515288,103822132,rebecca.nevin,1946584,2024-10-17 02:44:09 UTC,1650947495431285770,"{""yes"": 1, ""aggregation_version"": ""4.1.0""}",T0


## 5. Aggregate the annotations
Sort by unique subject ID and then unique tasks. Find the most recent classification for each user ID, and uses the Zooniverse consensus builder to look through all user classifications and build consensus.

In [56]:
aggregated_data = aggregate_data(extracted_data)

In [57]:
aggregated_data

,most_likely,num_votes,agreement,aggregation_version,subject_id,task,rubin_id
0,no,1,1.0,4.1.0,103822131,T0,1567965153859768169
1,no,1,1.0,4.1.0,103822132,T0,1650947495431285770
2,no,1,1.0,4.1.0,103822133,T0,1651448872733547971
3,no,1,1.0,4.1.0,103822134,T0,1651536833663756158
4,no,1,1.0,4.1.0,103822135,T0,1651325727431231924


## 6. Next steps and additional resources
You are now done! Congratulations!
Next steps could include joining the above table with other LSST data using the `rubin_id` column, which is either objectId or diaobjectId.

Additional resources include the Zooniverse team's resources to run panoptes through python (https://github.com/zooniverse/panoptes-python-client/tree/master), which provides high level access to the Zooniverse API in order to manage projects via python.

For examples of how to work with the data exports, see our Data Digging code repository or use our Panoptes Aggregation python package.
https://github.com/zooniverse/Data-digging, https://github.com/zooniverse/aggregation-for-caesar